In [1]:
from transformers import pipeline
model = pipeline('text-classification', model='../banking_en', device = 0)

/home/abderrahman.skiredj/.conda/envs/skiredj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-08 15:39:00.632743: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 15:39:00.670007: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 15:39:00.670049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 15:39:00.671628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register facto

In [3]:
classified = model(["i want to close my account", "i wanna activate my card", "how to shut down my card"])
classified

[{'label': 'delete_account', 'score': 0.8652453422546387},
 {'label': 'activate_my_card', 'score': 0.7381851673126221},
 {'label': 'deactivate_my_card', 'score': 0.6246784925460815}]

In [13]:
from collections import Counter

def compute_metrics(ground_truth, predictions):
    unique_labels = set(ground_truth + predictions)
    true_positives = Counter()
    false_positives = Counter()
    false_negatives = Counter()

    for true_label, predicted_label in zip(ground_truth, predictions):
        if true_label == predicted_label:
            true_positives[true_label] += 1
        else:
            false_positives[predicted_label] += 1
            false_negatives[true_label] += 1

    precision = {}
    recall = {}
    f1_score = {}

    for label in unique_labels:
        precision[label] = true_positives[label] / (true_positives[label] + false_positives[label]) if (true_positives[label] + false_positives[label]) else 0
        recall[label] = true_positives[label] / (true_positives[label] + false_negatives[label]) if (true_positives[label] + false_negatives[label]) else 0
        f1_score[label] = 2 * (precision[label] * recall[label]) / (precision[label] + recall[label]) if (precision[label] + recall[label]) else 0

    macro_precision = sum(precision.values()) / len(unique_labels)
    macro_recall = sum(recall.values()) / len(unique_labels)
    macro_f1_score = sum(f1_score.values()) / len(unique_labels)
    accuracy = sum(true_positives.values()) / len(ground_truth)

    return accuracy, macro_precision, macro_recall, macro_f1_score

# Example usage:
ground_truth = ['a', 'b', 'c', 'a', 'b', 'c', "d", "d"]
predictions = ['a', 'b', 'c', 'c', 'b', 'a', "e", "d"]
accuracy, precision, recall, f1_score = compute_metrics(ground_truth, predictions)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}")


Accuracy: 0.625
Precision: 0.6
Recall: 0.5
F1 Score: 0.5333333333333333


In [5]:
import pandas as pd
df = pd.read_excel('test_set_darija_8fev24_translated_eng.xlsx')

In [7]:
df

,text,label,language,translated
0,هادشي واعر,general_positive_feedback,ar,This is naked.
1,كيفاش نصلح طابعة عالقة؟,oodoos,ar,How do I fix a stuck printer?
2,واش نقدر نلغي طلبية لشهادة ايداع جديدة؟,cancel_order,ar,Can I cancel an order for a new deposit certif...
3,عمليات مشبوهة فحسابي.,compromised_card,ar,There are suspicious transactions on my account.
4,تأجل الماتش، شنو لازم ندير؟,get_refund,ar,The flight has been postponed. What should I do?
...,...,...,...,...
357,تقدر تسول الوكيل على ساعات دعم الزبائن؟,contact_customer_service,ar,Can you ask the agent about the customer servi...
358,كيفاش نعرف بلي تأميني مازال ساري؟,insurance,ar,How do I know if my insurance is still valid?
359,بغيت نعدل المعلومات فبروفيل ديالي، كيفاش نديرها؟,edit_account,ar,I want to change the information in my profile...
360,تقدر تحاول تواجه مشاكلي بلا ما تعطيني جوابات ج...,general_negative_feedback,ar,Can you try to solve my problems without givin...


In [9]:
ground_truths = df['label'].tolist()
df['translated'] = df['translated'].astype(str)
dar_translated_to_eng = df['translated'].tolist()

In [ ]:
df['translated'] = df['translated'].astype(str)
dar_translated_to_eng = df['translated'].tolist()

In [10]:
from transformers import pipeline
from tqdm import tqdm
dar_translated_classified = model(dar_translated_to_eng)

In [11]:
dar_translated_classified

[{'label': 'general_negative_feedback', 'score': 0.45667344331741333},
 {'label': 'oodoos', 'score': 0.7891215085983276},
 {'label': 'cancel_order', 'score': 0.8341993093490601},
 {'label': 'compromised_card', 'score': 0.5005671977996826},
 {'label': 'cancel_transfer', 'score': 0.39582332968711853},
 {'label': 'oodoos', 'score': 0.7863854765892029},
 {'label': 'cancel_order', 'score': 0.8344230055809021},
 {'label': 'contact_customer_service', 'score': 0.7687543034553528},
 {'label': 'get_refund', 'score': 0.8983446955680847},
 {'label': 'contact_customer_service', 'score': 0.8914391398429871},
 {'label': 'card_swallowed', 'score': 0.5294733047485352},
 {'label': 'fee', 'score': 0.8302009105682373},
 {'label': 'card_swallowed', 'score': 0.8206911683082581},
 {'label': 'contact_customer_service', 'score': 0.8878089785575867},
 {'label': 'change_order', 'score': 0.8238795399665833},
 {'label': 'delete_account', 'score': 0.8654446005821228},
 {'label': 'compromised_card', 'score': 0.76954

In [ ]:
ground_truths

In [14]:
intent_pred_dar = [x['label'] for x in dar_translated_classified]
accuracy, precision, recall, f1_score = compute_metrics(ground_truths, intent_pred_dar)
accuracy, precision, recall, f1_score

(0.8397790055248618,
 0.8570776599704618,
 0.8371269386072018,
 0.8370529358040776)

# test on arabic translated to eng

In [16]:
import pandas as pd
df = pd.read_excel('test_ar_translated_to_eng.xlsx')
ground_truths = df['intent'].tolist()
df['translated_from_ar'] = df['translated_from_ar'].astype(str)
ar_translated_to_eng = df['translated_from_ar'].tolist()
ar_translated_classified = model(ar_translated_to_eng)
intent_pred_ar = [x['label'] for x in ar_translated_classified]
accuracy, precision, recall, f1_score = compute_metrics(ground_truths, intent_pred_ar)
accuracy, precision, recall, f1_score

(0.8922651933701657,
 0.9070447297227485,
 0.8943748594242016,
 0.8903723297277396)